## Chapter 13: Deployment Fundamentals with Aspire

So far, we’ve focused on the **developer inner loop**—running your application locally with the AppHost, using containers for dependencies, and iterating quickly. But eventually, you need to deploy your application to a production environment. In the cloud‑native world, that environment is often **Kubernetes**.

Deploying a multi‑service application to Kubernetes manually is complex: you need to write YAML for Deployments, Services, ConfigMaps, Secrets, and Ingresses. You have to ensure that connection strings and environment variables are correctly set, and that services can discover each other. Aspire simplifies this by generating a **manifest** that describes your entire application model. Tools like `aspirate` (a community project) can then transform that manifest into Kubernetes‑ready YAML.

In this chapter, you’ll learn:

- What the Aspire manifest is and how to generate it.
- How to use the `aspirate` tool to create Kubernetes manifests.
- How the Aspire resource model maps to Kubernetes concepts.
- How to deploy your application to a local Kubernetes cluster (e.g., Docker Desktop).
- How to customize the generated output for production needs.

By the end, you’ll be able to take your Aspire application and deploy it to a Kubernetes cluster with minimal manual YAML writing.

---

### 13.1 From AppHost to Production

The AppHost is great for development, but it’s not a production orchestrator. In production, you need:

- **Resilience**: automatic restarts, scaling, self‑healing.
- **Service discovery** via Kubernetes DNS.
- **Configuration management** using ConfigMaps and Secrets.
- **Ingress** for external traffic.
- **Persistent volumes** for stateful services.

Kubernetes provides all of this. However, writing Kubernetes YAML by hand is error‑prone and tedious. Aspire’s approach is to **export** the application model from the AppHost into a standard format (the Aspire manifest). Then, tools can read that manifest and generate environment‑specific Kubernetes YAML.

The Aspire manifest is a JSON file that lists all resources, their dependencies, and how they are configured (environment variables, volumes, etc.). It is created by running the AppHost with a special publisher.

---

### 13.2 Generating the Aspire Manifest

To generate the manifest, run the AppHost project with the `--publisher manifest` flag and specify an output path.

```bash
cd MyAspireApp.AppHost
dotnet run -- --publisher manifest --output-path ../aspire-manifest.json
```

This executes the AppHost, but instead of starting the application, it runs a **manifest publisher** that walks the resource graph and writes a JSON representation to the specified file.

The manifest includes:

- All resources (projects, containers, executables).
- Their dependencies (via `WithReference`).
- Environment variables and connection strings.
- Volumes and bind mounts.
- Health checks and other annotations.

Example snippet (simplified):

```json
{
  "resources": {
    "apiservice": {
      "type": "project.v0",
      "path": "../MyAspireApp.ApiService/MyAspireApp.ApiService.csproj",
      "env": {
        "OTEL_EXPORTER_OTLP_ENDPOINT": "http://localhost:4317"
      },
      "bindings": {
        "http": { "scheme": "http", "protocol": "tcp", "port": 5001 },
        "https": { "scheme": "https", "protocol": "tcp", "port": 5002 }
      }
    },
    "postgres": {
      "type": "container.v0",
      "image": "postgres:16.2",
      "env": {
        "POSTGRES_PASSWORD": "{postgres-password}"
      },
      "volumes": [
        { "name": "postgres-data", "target": "/var/lib/postgresql/data" }
      ]
    }
  }
}
```

This manifest is the source of truth for deployment tools.

---

### 13.3 Introducing `aspirate`

`aspirate` (often stylized as `aspir8`) is a community‑developed tool that consumes the Aspire manifest and generates Kubernetes YAML. It’s written in .NET and can be installed as a global tool.

#### 13.3.1 Installing `aspirate`

```bash
dotnet tool install -g aspirate
```

Verify installation:

```bash
aspirate --version
```

#### 13.3.2 Basic Usage

Navigate to the directory containing your `aspire-manifest.json`. Then run:

```bash
aspirate generate
```

This will:

1. Read the manifest.
2. For each resource, create corresponding Kubernetes manifests (Deployment, Service, ConfigMap, Secret, etc.).
3. Output the YAML files into a folder (default `./aspirate-output`).

You can then apply these manifests to your Kubernetes cluster with `kubectl apply -f aspirate-output/`.

#### 13.3.3 What `aspirate` Generates

Let’s examine the mapping:

- **Project resources** become Kubernetes **Deployments** (with the container image built from the project) and a corresponding **Service**.
- **Container resources** (like PostgreSQL) become **Deployments** (if they are meant to run as pods) or maybe **StatefulSets** if they need stable storage. `aspirate` typically generates Deployments with a single replica.
- **Connection strings** and environment variables become either **ConfigMaps** (for non‑secret data) or **Secrets** (for values marked as secret). `aspirate` uses the parameter system to determine what’s secret.
- **Volumes** become **PersistentVolumeClaims** or emptyDir volumes.
- **Dependencies** are expressed via `dependsOn` in the Deployment specs, but Kubernetes doesn’t enforce startup order; you may need init containers or wait logic.

`aspirate` also generates a **Kustomization** file if you use kustomize, making it easy to overlay environment‑specific changes.

---

### 13.4 Preparing for Deployment

Before we generate manifests, we need to make sure our application is ready for Kubernetes.

#### 13.4.1 Building Container Images

Project resources in Aspire need to be built into container images. `aspirate` can handle this for you. It will:

- Use `dotnet publish` to compile the project.
- Build a Docker image using a default Dockerfile (or one you provide).
- Tag the image with a name derived from the project.

You need a container registry to push the images to. For local development, you can use a local registry like Docker Desktop’s built‑in one, or you can load images directly into your cluster (if using kind or minikube). `aspirate` supports both: it can build images and push them to a registry, or it can generate YAML that references images you’ve built manually.

We’ll assume you’re using a local Kubernetes cluster (like Docker Desktop) and you want to load images directly. Docker Desktop’s Kubernetes can use images built with the Docker daemon without a registry.

#### 13.4.2 Setting Parameters and Secrets

In your AppHost, you used parameters for secrets (like database passwords, Keycloak client secrets). In the manifest, these appear as placeholders like `{postgres-password}`. `aspirate` will prompt you for these values when generating manifests, or you can provide them via a `aspirate.json` configuration file.

You can create an `aspirate.json` in the same directory as the manifest to pre‑fill values:

```json
{
  "parameters": {
    "postgres-password": "mysecretpassword",
    "keycloak-admin-password": "admin123"
  }
}
```

For production, you’d likely store these in a secure vault and inject them via environment variables or external secrets.

---

### 13.5 Hands‑on: Deploying to Local Kubernetes

Let’s deploy our e‑commerce application to a local Kubernetes cluster. We’ll use Docker Desktop’s built‑in Kubernetes (enable it in Docker Desktop settings). Alternatively, you can use Minikube or K3s.

#### Step 1: Enable Kubernetes in Docker Desktop

In Docker Desktop, go to Settings → Kubernetes → Enable Kubernetes. Apply and restart. Ensure `kubectl` is installed and configured to point to this cluster.

#### Step 2: Generate the Aspire Manifest

From the AppHost directory:

```bash
dotnet run -- --publisher manifest --output-path ../aspire-manifest.json
```

This creates `aspire-manifest.json` in the solution root.

#### Step 3: Run `aspirate generate`

Navigate to the solution root (where the manifest is). Run:

```bash
aspirate generate
```

You’ll be prompted for any parameters that weren’t provided. For example:

- `postgres-password`: enter a password.
- `keycloak-admin-password`: enter a password.

`aspirate` will then:

- Build Docker images for each project (using `docker build`). It will look for a Dockerfile in each project directory. If none exists, you need to create one. The Aspire template doesn’t include Dockerfiles by default because it runs projects as processes. For Kubernetes, we need container images. Let's create a simple Dockerfile for the API service.

Create a file `MyAspireApp.ApiService/Dockerfile`:

```dockerfile
FROM mcr.microsoft.com/dotnet/aspnet:8.0 AS base
WORKDIR /app
EXPOSE 80
EXPOSE 443

FROM mcr.microsoft.com/dotnet/sdk:8.0 AS build
WORKDIR /src
COPY ["MyAspireApp.ApiService/MyAspireApp.ApiService.csproj", "MyAspireApp.ApiService/"]
RUN dotnet restore "MyAspireApp.ApiService/MyAspireApp.ApiService.csproj"
COPY . .
WORKDIR "/src/MyAspireApp.ApiService"
RUN dotnet build "MyAspireApp.ApiService.csproj" -c Release -o /app/build

FROM build AS publish
RUN dotnet publish "MyAspireApp.ApiService.csproj" -c Release -o /app/publish

FROM base AS final
WORKDIR /app
COPY --from=publish /app/publish .
ENTRYPOINT ["dotnet", "MyAspireApp.ApiService.dll"]
```

Repeat for the web frontend and worker service. Adjust paths accordingly (since the Dockerfile is in the project folder, the context should be the solution root when building). `aspirate` will handle this.

Now run `aspirate generate` again. This time it will build images. The images will be tagged like `apiservice:latest`, `webfrontend:latest`, etc.

`aspirate` will output Kubernetes YAML in `./aspirate-output/`. Examine the files:

- `apiservice-deployment.yaml`
- `apiservice-service.yaml`
- `postgres-deployment.yaml` (if you have PostgreSQL)
- `postgres-service.yaml`
- Secrets and ConfigMaps.

#### Step 4: Apply the Manifests

```bash
kubectl apply -f aspirate-output/
```

This creates all resources in the default namespace. You can specify a namespace with `-n` or add a namespace to the YAML.

#### Step 5: Verify Deployment

Check pods:

```bash
kubectl get pods
```

You should see all pods running. If any are in `ImagePullBackoff` or `CrashLoopBackOff`, investigate with `kubectl logs <pod>`.

To access the web frontend, we need to expose it. By default, `aspirate` creates a ClusterIP service. For local access, you can port‑forward:

```bash
kubectl port-forward service/webfrontend 8080:80
```

Then open `http://localhost:8080`. You should see your e‑commerce frontend.

#### Step 6: Test the Application

Try to view products, place an order, etc. Note that service discovery in Kubernetes works differently: the web frontend will try to reach `http://apiservice`. In Kubernetes, that resolves to the `apiservice` service (thanks to DNS). So as long as the service is named `apiservice`, it will work.

Check that the worker is processing messages, and that PostgreSQL is accessible.

#### Step 7: Clean Up

When done, delete the resources:

```bash
kubectl delete -f aspirate-output/
```

---

### 13.6 Customizing the Output

`aspirate` supports customization via an `aspirate.json` configuration file. You can specify:

- Container registry (e.g., `myregistry.azurecr.io`) to push images.
- Namespace for all resources.
- Ingress settings to expose services externally.
- Resource limits and requests.
- Environment‑specific overrides.

Example `aspirate.json`:

```json
{
  "containerRegistry": "myregistry.azurecr.io",
  "containerRepositoryPrefix": "myapp",
  "imagePullPolicy": "Always",
  "namespace": "myapp-prod",
  "ingress": {
    "enabled": true,
    "host": "myapp.example.com",
    "annotations": {
      "kubernetes.io/ingress.class": "nginx"
    }
  }
}
```

Then run `aspirate generate` again; it will incorporate these settings.

---

### 13.7 Production Considerations

Deploying to a production Kubernetes cluster involves more steps:

- Use a proper container registry (Azure Container Registry, Docker Hub, etc.).
- Set up TLS certificates for ingress (e.g., with cert‑manager).
- Configure external database services (e.g., Azure Database for PostgreSQL) instead of running PostgreSQL in Kubernetes.
- Use managed identity or secrets from a vault (Azure Key Vault, HashiCorp Vault) instead of Kubernetes Secrets.
- Set up monitoring (Prometheus, Grafana) and logging (ELK, Azure Monitor).
- Implement horizontal pod autoscaling based on metrics.

Aspire’s manifest and `aspirate` provide a solid foundation, but you’ll need to tailor the generated YAML or use overlays (kustomize) for production‑specific changes.

---

### 13.8 Summary

In this chapter, you learned how to take your Aspire application from development to a Kubernetes cluster:

- The Aspire manifest captures your application model in a JSON file.
- The `aspirate` tool consumes the manifest and generates Kubernetes YAML.
- You can deploy to a local Kubernetes cluster for testing.
- `aspirate` handles building container images and mapping resources to Kubernetes concepts.
- Customization via `aspirate.json` allows you to adapt to different environments.

With these skills, you can deploy your Aspire application to any Kubernetes cluster. In the next chapter, we’ll explore **Deploying to the Cloud – Azure Container Apps**, a serverless container platform that integrates deeply with Aspire.

---

**Exercises**

1. Create Dockerfiles for all your projects and successfully generate Kubernetes manifests with `aspirate`. Deploy to a local cluster and verify the application works.
2. Modify the `aspirate.json` to include an ingress resource and deploy with an ingress controller (e.g., nginx‑ingress). Access the application via the ingress host (you may need to edit `/etc/hosts`).
3. Experiment with scaling the API deployment to 3 replicas. Observe how the web frontend load‑balances using Kubernetes service (which does round‑robin by default).
4. Add resource requests and limits to the generated manifests and verify they are applied.

In Chapter 14, we’ll cover **Deploying to the Cloud – Azure Container Apps**, using the Azure Developer CLI (AZD) for a seamless deployment experience.